# Project 2

Dataset 1: 20180101-20221231 CAISO Real-time Price (every 5 minutes)

Dataset 2: Historical Load in CAISO 2019-2022 (daily)

Target: Comapre SP15 Daily Data in 2022

## Dataset 1

In [64]:
import pandas as pd

prices = pd.read_csv("20180101-20221231 CAISO Real-time Price.csv")
print(prices.head())


                   Date     price      hub
0  1/1/2018 12:00:00 AM  38.89844  TH_NP15
1  1/1/2018 12:00:00 AM  38.85151  TH_SP15
2  1/1/2018 12:00:00 AM  38.51520  TH_ZP26
3  1/1/2018 12:05:00 AM  53.66772  TH_NP15
4  1/1/2018 12:05:00 AM  53.60298  TH_SP15


In [65]:
print(prices['hub'].unique())

['TH_NP15' 'TH_SP15' 'TH_ZP26']


In [67]:
prices = prices.copy()
# Filter SP15 data, which is called "TH_SP15" here
SP15prices = prices[prices['hub'] == "TH_SP15"].copy()

print(SP15prices.head())


                    Date     price      hub
1   1/1/2018 12:00:00 AM  38.85151  TH_SP15
4   1/1/2018 12:05:00 AM  53.60298  TH_SP15
7   1/1/2018 12:10:00 AM  53.60298  TH_SP15
10  1/1/2018 12:15:00 AM  52.87035  TH_SP15
13  1/1/2018 12:20:00 AM  53.19333  TH_SP15


In [ ]:
# ensure datetime
SP15prices['Date'] = pd.to_datetime(
    SP15prices['Date'],
    format="%m/%d/%Y %I:%M:%S %p"
)

# set index
SP15 = SP15prices.set_index('Date')

# resample to daily average price
daily_price = SP15['price'].resample('D').mean().reset_index()

daily_price.head()


        Date   hub      price
96   2018-01  SP15  34.476337
97   2018-02  SP15  38.026694
98   2018-03  SP15  31.501154
99   2018-04  SP15  24.972061
100  2018-05  SP15  21.706315


## Dataset 2

In [16]:
solgen = pd.read_csv("SolGen18_21.csv")
print(solgen.head())


  Trading Hub    Date  Daily Solar Generation (MW)
0        NP15  1/1/18                     51627.37
1        NP15  1/2/18                     21734.78
2        NP15  1/3/18                     22806.95
3        NP15  1/4/18                     44141.84
4        NP15  1/5/18                     53282.33


In [20]:
# filter out rows with missing values
solgen = solgen.dropna()
# convert 'Date' column to datetime
solgen['Date'] = pd.to_datetime(solgen['Date'], format="%Y-%m-%d", errors="coerce")
# filter from 2018-01-01 to 2020-12-31
year1820 = solgen[(solgen['Date'] >= "2018-01-01") & (solgen['Date'] <= "2020-12-31")]

print(year1820.head())


  Trading Hub       Date  Daily Solar Generation (MW)
0        NP15 2018-01-01                     51627.37
1        NP15 2018-01-02                     21734.78
2        NP15 2018-01-03                     22806.95
3        NP15 2018-01-04                     44141.84
4        NP15 2018-01-05                     53282.33


In [22]:
print(year1820['Trading Hub'].unique())

['NP15' 'SP15']


In [ ]:
# Filter SP15 solar generation data
sp15solar = year1820[year1820['Trading Hub'] == "SP15"]
print(sp15solar.head())

    Trading Hub       Date  Daily Solar Generation (MW)
122        SP15 2018-01-01                    371875.94
123        SP15 2018-01-02                    295079.05
124        SP15 2018-01-03                    201788.24
125        SP15 2018-01-04                    379863.10
126        SP15 2018-01-05                    410281.44


In [56]:
# Convert Daily data to monthly data
d2 = sp15solar.resample('ME', on='Date')['Daily Solar Generation (MW)'].mean().reset_index()
d2['Date'] = d2['Date'].dt.to_period('M')
d2 = d2.dropna()

print(d2.head())

       Date  Daily Solar Generation (MW)
0   2018-01                397250.757097
5   2018-06                836733.983333
12  2019-01                361630.392258
17  2019-06                877997.715333
24  2020-01                479701.018710


## Merging Data

In [58]:
merged = pd.merge(d1, d2, on='Date', how='inner')

print(merged.head())
print(merged.shape)


      Date   hub      price  Daily Solar Generation (MW)
0  2018-01  SP15  34.476337                397250.757097
1  2018-06  SP15  26.002650                836733.983333
2  2019-01  SP15  38.855867                361630.392258
3  2019-06  SP15  35.803951                877997.715333
4  2020-01  SP15  25.342313                479701.018710
(6, 4)
